In [1]:
import os
import numpy as np
import cv2
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms


class CoronarySmallDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = os.listdir(image_dir)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        SIZE = 256
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        

        image = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
        # image = cv2.cvtColor(image, cv2.COLOR_RGBA2RGB)
        image = cv2.resize(image, (SIZE, SIZE), cv2.INTER_NEAREST)
        
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        # mask = cv2.cvtColor(mask, cv2.COLOR_RGBA2RGB)
        mask = cv2.resize(mask, (SIZE, SIZE), cv2.INTER_NEAREST)
        
        if self.transform:
            image = self.transform(image)
            # mask = self.transform(mask)
            mask = torch.tensor(mask, dtype=torch.long)
        
        return image, mask


transform = transforms.Compose([
    transforms.ToTensor()
])

train_image_dir = '..\images\images_right\images_train\input'
train_mask_dir = '..\images\images_right\images_train\output'
val_image_dir = '..\images\images_right\images_val\input'
val_mask_dir = '..\images\images_right\images_val\output'
test_image_dir = '..\images\images_right\images_test\input'
test_mask_dir = '..\images\images_right\images_test\output'

train_dataset = CoronarySmallDataset(train_image_dir, train_mask_dir, transform=transform)
val_dataset = CoronarySmallDataset(val_image_dir, val_mask_dir, transform=transform)
test_dataset = CoronarySmallDataset(test_image_dir, test_mask_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)


In [2]:
from converter_RGB import convert_int_to_RGB
from large_RGB_model import UNet


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet()
model = model.to(device)

# model.load_state_dict(torch.load('model_binary_6.pth'))
# model.eval()

def show_image(type, image_name):
    dir = f"..\images\images_right\images_test\{type}"
    print(dir)
    if type == 'output':
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        # img = img[:, :, 2]
        print("Oryginalnie: ", np.unique(img))
        
        print("Po resize:  ", np.unique(cv2.resize(img, (256, 256))))
        # print(img)
        img = convert_int_to_RGB(img)
        img = cv2.cvtColor(img, cv2.COLOR_BGRA2RGB)
    else:
        img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
        img = img * 7
    # img = cv2.resize(img, (256, 256))
    # np.set_printoptions(threshold=np.inf)
    cv2.imshow(type, img)

def predict(image_name):
    dir = '..\images\images_right\images_test\input'
    img = cv2.imread(os.path.join(dir, image_name), cv2.IMREAD_UNCHANGED)
    img = cv2.resize(img, (256, 256))
    img = transforms.ToTensor()(img).unsqueeze(0).to(device)

    with torch.no_grad():
        pred = model.predict(img)
        pred = pred.squeeze().cpu().numpy()
    print("Z sieci:    ", np.unique(pred))

    pred_image = convert_int_to_RGB(pred)
    pred_image = cv2.resize(pred_image, (512, 512))
    pred_image = cv2.cvtColor(pred_image, cv2.COLOR_BGR2RGB)
    cv2.imshow('pred', pred_image)
    cv2.waitKey(0)
   

In [18]:
# image_name = "131aedfhs6pnf1fvtvp49mhdb2fucqzo22_29.png"
# image_name = "131aedfhs6pnf1fvtvp49mld7mqexnz322_36.png"
# image_name = "131aedfhs6pnf1fvtvp49mia892s56cf22_28.png"
# image_name = "131aedfhs6pnf1fvtvp49juwu7plj9dv22_40.png"
# image_name = "13c2ur549vohc0jat2dvu3xs7q1_18.png"
image_name = "131aedfhs6pnf1fvtvp49mi8hml81goj22_37.png"

..\images\images_test\input
..\images\images_test\output
Oryginalnie:  [ 0  5  6  7  8  9 10 11 13 15 16 17]
Po resize:   [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17]
Z sieci:     [ 0  5  6  7  8  9 13 15 17 18]


In [3]:
from sklearn.metrics import precision_score, recall_score, f1_score


def compute_accuracy(model, data_loader, device):
    model.eval()
    correct = 0
    correct_clear = 0
    total = 0
    total_clear = 0

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            correct += (preds == masks).sum().item()
            correct_clear += (preds[masks != 0] == masks[masks != 0]).sum().item()
            total += masks.numel()
            total_clear += masks[masks != 0].numel()

    accuracy = correct / total
    accuracy_clear = correct_clear / total_clear
    return accuracy, accuracy_clear

In [4]:
# Do poprawy, teraz nie ma sensu
# Musiałoby dla każdej klasu osobno liczyć
def compute_precision_recall_f1(model, data_loader, device, num_classes=28):
    model.eval()
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for images, masks in data_loader:
            images = images.to(device)
            masks = masks.to(device)

            preds = model.predict(images)
            
            all_preds.append(preds.view(-1).cpu().numpy())
            all_targets.append(masks.view(-1).cpu().numpy())

    all_preds = np.concatenate(all_preds)
    all_targets = np.concatenate(all_targets)

    TP = np.zeros(num_classes)
    FP = np.zeros(num_classes)
    FN = np.zeros(num_classes)

    for i in range(num_classes):
        TP[i] = ((all_preds == i) & (all_targets == i)).sum()
        FP[i] = ((all_preds == i) & (all_targets != i)).sum()
        FN[i] = ((all_preds != i) & (all_targets == i)).sum()
    
    TP[0], FP[0], FN[0] = 0, 0, 0
    # print(TP, FP, FN)

    precision = TP / (TP + FP)
    recall = TP / (TP + FN)
    f1 = 2 * precision * recall / (precision + recall)
    # print(TP.sum(), FP.sum(), FN.sum())

    precision_weighted = TP.sum() / (TP.sum() + FP.sum())
    recall_weighted = TP.sum() / (TP.sum() + FN.sum())
    f1_weighted = 2 * precision_weighted * recall_weighted / (precision_weighted + recall_weighted)

    return precision, recall, f1, precision_weighted, recall_weighted, f1_weighted

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_binary_right1.pth'))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

Train Accuracy: 98.53%
Train Accuracy Clear: 63.36%

Validation Accuracy: 98.48%
Validation Accuracy Clear: 62.90%


In [7]:
model.load_state_dict(torch.load('model_binary_right1.pth'))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 83.41%, Recall: 75.06%, F1: 79.01%
Class 2: Precision: 66.74%, Recall: 69.58%, F1: 68.13%
Class 3: Precision: 56.13%, Recall: 67.23%, F1: 61.18%
Class 4: Precision: 65.18%, Recall: 60.71%, F1: 62.86%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 7: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 18: Precision: 0.00%, Recall: 0.00%, F

C:\Users\Szymon\AppData\Local\Temp\ipykernel_19340\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\Szymon\AppData\Local\Temp\ipykernel_19340\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [8]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_binary_right2.pth'))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

Train Accuracy: 98.44%
Train Accuracy Clear: 61.29%

Validation Accuracy: 98.45%
Validation Accuracy Clear: 62.33%


In [9]:
model.load_state_dict(torch.load('model_binary_right2.pth'))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 82.86%, Recall: 67.94%, F1: 74.66%
Class 2: Precision: 64.05%, Recall: 69.56%, F1: 66.69%
Class 3: Precision: 58.29%, Recall: 72.16%, F1: 64.49%
Class 4: Precision: 58.56%, Recall: 51.78%, F1: 54.96%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 7: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 18: Precision: 0.00%, Recall: 0.00%, F

C:\Users\Szymon\AppData\Local\Temp\ipykernel_19340\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\Szymon\AppData\Local\Temp\ipykernel_19340\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)
C:\Users\Szymon\AppData\Local\Temp\ipykernel_19340\3879195863.py:35: RuntimeWarning: invalid value encountered in divide
  f1 = 2 * precision * recall / (precision + recall)


In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_binary_right3.pth'))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

Train Accuracy: 98.72%
Train Accuracy Clear: 68.24%

Validation Accuracy: 98.73%
Validation Accuracy Clear: 68.90%


In [11]:
model.load_state_dict(torch.load('model_binary_right3.pth'))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 86.83%, Recall: 72.17%, F1: 78.82%
Class 2: Precision: 73.44%, Recall: 64.10%, F1: 68.45%
Class 3: Precision: 69.21%, Recall: 83.95%, F1: 75.87%
Class 4: Precision: 67.04%, Recall: 75.49%, F1: 71.02%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 7: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 18: Precision: 0.00%, Recall: 0.00%, F

C:\Users\Szymon\AppData\Local\Temp\ipykernel_19340\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\Szymon\AppData\Local\Temp\ipykernel_19340\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_binary_right4.pth'))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

Train Accuracy: 98.71%
Train Accuracy Clear: 67.95%

Validation Accuracy: 98.70%
Validation Accuracy Clear: 68.15%


In [13]:
model.load_state_dict(torch.load('model_binary_right4.pth'))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 93.89%, Recall: 61.42%, F1: 74.26%
Class 2: Precision: 66.68%, Recall: 70.56%, F1: 68.57%
Class 3: Precision: 70.44%, Recall: 82.01%, F1: 75.79%
Class 4: Precision: 69.14%, Recall: 79.91%, F1: 74.13%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 7: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 18: Precision: 0.00%, Recall: 0.00%, F

C:\Users\Szymon\AppData\Local\Temp\ipykernel_19340\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\Szymon\AppData\Local\Temp\ipykernel_19340\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)


In [16]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = UNet().to(device)
model.load_state_dict(torch.load('model_binary_right5.pth'))

train_accuracy = compute_accuracy(model, train_loader, device)
val_accuracy = compute_accuracy(model, val_loader, device)

print(f'Train Accuracy: {train_accuracy[0] * 100 :.2f}%')
print(f'Train Accuracy Clear: {train_accuracy[1] * 100 :.2f}%')
print()
print(f'Validation Accuracy: {val_accuracy[0] * 100 :.2f}%')
print(f'Validation Accuracy Clear: {val_accuracy[1] * 100 :.2f}%')

Train Accuracy: 98.81%
Train Accuracy Clear: 70.33%

Validation Accuracy: 98.78%
Validation Accuracy Clear: 70.21%


In [17]:
model.load_state_dict(torch.load('model_binary_right5.pth'))
precision, recall, f1, precision_weighted, recall_weighted, f1_weighted = compute_precision_recall_f1(model, val_loader, device)

if np.isnan(precision).any():
    precision = np.nan_to_num(precision)
if np.isnan(recall).any():
    recall = np.nan_to_num(recall)
if np.isnan(f1).any():
    f1 = np.nan_to_num(f1)

for i in range(28):
    print(f'Class {i}: Precision: {precision[i] * 100:.2f}%, Recall: {recall[i] * 100:.2f}%, F1: {f1[i] * 100:.2f}%')

print(f'Average Precision: {np.mean(precision) * 100:.3f}%')
print(f'Average Recall: {np.mean(recall) * 100:.3f}%')
print(f'Average F1: {np.mean(f1) * 100:.3f}%')
print()
print(f'Weighted Precision: {precision_weighted * 100:.3f}%')
print(f'Weighted Recall: {recall_weighted * 100:.3f}%')
print(f'Weighted F1: {f1_weighted * 100:.3f}%')

Class 0: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 1: Precision: 92.02%, Recall: 69.39%, F1: 79.12%
Class 2: Precision: 71.47%, Recall: 69.41%, F1: 70.42%
Class 3: Precision: 76.90%, Recall: 79.72%, F1: 78.29%
Class 4: Precision: 74.63%, Recall: 85.41%, F1: 79.66%
Class 5: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 6: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 7: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 8: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 9: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 10: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 11: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 12: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 13: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 14: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 15: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 16: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 17: Precision: 0.00%, Recall: 0.00%, F1: 0.00%
Class 18: Precision: 0.00%, Recall: 0.00%, F

C:\Users\Szymon\AppData\Local\Temp\ipykernel_19340\3879195863.py:33: RuntimeWarning: invalid value encountered in divide
  precision = TP / (TP + FP)
C:\Users\Szymon\AppData\Local\Temp\ipykernel_19340\3879195863.py:34: RuntimeWarning: invalid value encountered in divide
  recall = TP / (TP + FN)
